## Введение в семантическое чанкование
Чанкование текста - важный этап в Retrieval-Augmented Generation (RAG), где большие тексты делятся на осмысленные сегменты для повышения точности поиска.
В отличие от чанкования фиксированной длины, семантическое чанкование разделяет текст на основе смысловой схожести между предложениями.

### Методы определения границ чанков:
- **Процентиль**: Находит X-й процентиль всех различий схожести и разделяет чанки там, где падение превышает это значение.
- **Стандартное отклонение**: Разделяет там, где схожесть падает более чем на X стандартных отклонений ниже среднего.
- **Межквартильный размах (IQR)**: Использует межквартильное расстояние (Q3 - Q1) для определения точек разделения.

Этот ноутбук реализует семантическое чанкование **с использованием метода процентилей** и оценивает его производительность на примере текста.

## Настройка окружения
Начнём с импорта необходимых библиотек.

In [1]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

## Извлечение текста из PDF-файла
Для реализации RAG нам сначала нужен источник текстовых данных. В данном случае мы извлекаем текст из PDF-файла с помощью библиотеки PyMuPDF.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    Извлекает текст из PDF-файла.

    Аргументы:
    pdf_path (str): Путь к PDF-файлу.

    Возвращает:
    str: Извлечённый текст из PDF.
    """
    # Открываем PDF-файл
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Инициализируем пустую строку для хранения извлечённого текста
    
    # Итерируемся по каждой странице в PDF
    for page in mypdf:
        # Извлекаем текст с текущей страницы и добавляем пробел
        all_text += page.get_text("text") + " "

    # Возвращаем извлечённый текст, очищенный от начальных/конечных пробелов
    return all_text.strip()

# Определяем путь к PDF-файлу
pdf_path = "data/AI_Information.pdf"

# Извлекаем текст из PDF-файла
extracted_text = extract_text_from_pdf(pdf_path)

# Выводим первые 500 символов извлечённого текста
print(extracted_text[:500])

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past f


## Настройка клиента OpenAI API
Инициализируем клиент OpenAI для генерации эмбеддингов и ответов.

In [ ]:
# Инициализируем клиент OpenAI с базовым URL и API ключом
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # Получаем API ключ из переменных окружения
)

## Создание эмбеддингов на уровне предложений
Разделяем текст на предложения и генерируем эмбеддинги.

In [4]:
def get_embedding(text, model="BAAI/bge-en-icl"):
    """
    Создаёт эмбеддинг для заданного текста с использованием OpenAI.

    Аргументы:
    text (str): Входной текст.
    model (str): Название модели эмбеддингов.

    Возвращает:
    np.ndarray: Вектор эмбеддинга.
    """
    response = client.embeddings.create(model=model, input=text)
    return np.array(response.data[0].embedding)

# Разделяем текст на предложения (базовое разделение)
sentences = extracted_text.split(". ")

# Генерируем эмбеддинги для каждого предложения
embeddings = [get_embedding(sentence) for sentence in sentences]

print(f"Generated {len(embeddings)} sentence embeddings.")

Generated 257 sentence embeddings.


## Вычисление различий схожести
Вычисляем косинусную схожесть между последовательными предложениями.

In [5]:
def cosine_similarity(vec1, vec2):
    """
    Вычисляет косинусную схожесть между двумя векторами.

    Аргументы:
    vec1 (np.ndarray): Первый вектор.
    vec2 (np.ndarray): Второй вектор.

    Возвращает:
    float: Косинусная схожесть.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Вычисляем схожесть между последовательными предложениями
similarities = [cosine_similarity(embeddings[i], embeddings[i + 1]) for i in range(len(embeddings) - 1)]

## Реализация семантического чанкования
Реализуем три различных метода для нахождения точек разделения.

In [6]:
def compute_breakpoints(similarities, method="percentile", threshold=90):
    """
    Вычисляет точки разделения для чанков на основе падений схожести.

    Аргументы:
    similarities (List[float]): Список оценок схожести между предложениями.
    method (str): 'percentile', 'standard_deviation' или 'interquartile'.
    threshold (float): Пороговое значение (процентиль для 'percentile', стандартные отклонения для 'standard_deviation').

    Возвращает:
    List[int]: Индексы, где должны происходить разделения чанков.
    """
    # Определяем пороговое значение на основе выбранного метода
    if method == "percentile":
        # Вычисляем X-й процентиль оценок схожести
        threshold_value = np.percentile(similarities, threshold)
    elif method == "standard_deviation":
        # Вычисляем среднее и стандартное отклонение оценок схожести
        mean = np.mean(similarities)
        std_dev = np.std(similarities)
        # Устанавливаем пороговое значение как среднее минус X стандартных отклонений
        threshold_value = mean - (threshold * std_dev)
    elif method == "interquartile":
        # Вычисляем первый и третий квартили (Q1 и Q3)
        q1, q3 = np.percentile(similarities, [25, 75])
        # Устанавливаем пороговое значение по правилу IQR для выбросов
        threshold_value = q1 - 1.5 * (q3 - q1)
    else:
        # Вызываем ошибку, если предоставлен недопустимый метод
        raise ValueError("Invalid method. Choose 'percentile', 'standard_deviation', or 'interquartile'.")

    # Определяем индексы, где схожесть падает ниже порогового значения
    return [i for i, sim in enumerate(similarities) if sim < threshold_value]

# Вычисляем точки разделения с использованием метода процентилей с порогом 90
breakpoints = compute_breakpoints(similarities, method="percentile", threshold=90)

## Разделение текста на семантические чанки
Разделяем текст на основе вычисленных точек разделения.

In [7]:
def split_into_chunks(sentences, breakpoints):
    """
    Разделяет предложения на семантические чанки.

    Аргументы:
    sentences (List[str]): Список предложений.
    breakpoints (List[int]): Индексы, где должно происходить разделение.

    Возвращает:
    List[str]: Список текстовых чанков.
    """
    chunks = []  # Инициализируем пустой список для хранения чанков
    start = 0  # Инициализируем начальный индекс

    # Итерируемся по каждой точке разделения для создания чанков
    for bp in breakpoints:
        # Добавляем чанк предложений от start до текущей точки разделения
        chunks.append(". ".join(sentences[start:bp + 1]) + ".")
        start = bp + 1  # Обновляем начальный индекс на следующее предложение после точки разделения

    # Добавляем оставшиеся предложения как последний чанк
    chunks.append(". ".join(sentences[start:]))
    return chunks  # Возвращаем список чанков

# Создаём чанки с помощью функции split_into_chunks
text_chunks = split_into_chunks(sentences, breakpoints)

# Выводим количество созданных чанков
print(f"Number of semantic chunks: {len(text_chunks)}")

# Выводим первый чанк для проверки результата
print("\nFirst text chunk:")
print(text_chunks[0])


Number of semantic chunks: 231

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings.


## Создание эмбеддингов для семантических чанков
Создаём эмбеддинги для каждого чанка для последующего поиска.

In [8]:
def create_embeddings(text_chunks):
    """
    Создаёт эмбеддинги для каждого текстового чанка.

    Аргументы:
    text_chunks (List[str]): Список текстовых чанков.

    Возвращает:
    List[np.ndarray]: Список векторов эмбеддингов.
    """
    # Генерируем эмбеддинги для каждого текстового чанка с помощью функции get_embedding
    return [get_embedding(chunk) for chunk in text_chunks]

# Создаём эмбеддинги чанков с помощью функции create_embeddings
chunk_embeddings = create_embeddings(text_chunks)

## Выполнение семантического поиска
Используем косинусную схожесть для поиска наиболее релевантных чанков.

In [9]:
def semantic_search(query, text_chunks, chunk_embeddings, k=5):
    """
    Находит наиболее релевантные текстовые чанки для запроса.

    Аргументы:
    query (str): Поисковый запрос.
    text_chunks (List[str]): Список текстовых чанков.
    chunk_embeddings (List[np.ndarray]): Список эмбеддингов чанков.
    k (int): Количество возвращаемых результатов.

    Возвращает:
    List[str]: Топ-k релевантных чанков.
    """
    # Генерируем эмбеддинг для запроса
    query_embedding = get_embedding(query)
    
    # Вычисляем косинусную схожесть между эмбеддингом запроса и каждым эмбеддингом чанка
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # Получаем индексы топ-k наиболее схожих чанков
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # Возвращаем топ-k наиболее релевантных текстовых чанков
    return [text_chunks[i] for i in top_indices]

In [10]:
# Загружаем валидационные данные из JSON-файла
with open('data/val.json') as f:
    data = json.load(f)

# Извлекаем первый запрос из валидационных данных
query = data[0]['question']

# Получаем топ 2 релевантных чанка
top_chunks = semantic_search(query, text_chunks, chunk_embeddings, k=2)

# Выводим запрос
print(f"Query: {query}")

# Выводим топ 2 наиболее релевантных текстовых чанка
for i, chunk in enumerate(top_chunks):
    print(f"Context {i+1}:\n{chunk}\n{'='*40}")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:

Explainable AI (XAI) 
Explainable AI (XAI) aims to make AI systems more transparent and understandable. Research in 
XAI focuses on developing methods for explaining AI decisions, enhancing trust, and improving 
accountability.
Context 2:

Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy.


## Генерация ответа на основе извлечённых чанков

In [11]:
# Определяем системный промт для AI-ассистента
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Генерирует ответ от AI модели на основе системного промта и пользовательского сообщения.

    Аргументы:
    system_prompt (str): Системный промт для управления поведением AI.
    user_message (str): Сообщение пользователя или запрос.
    model (str): Модель для генерации ответа. По умолчанию "meta-llama/Llama-2-7B-chat-hf".

    Возвращает:
    dict: Ответ от AI модели.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Создаём пользовательский промт на основе топовых чанков
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Генерируем AI ответ
ai_response = generate_response(system_prompt, user_prompt)

## Оценка ответа AI
Сравниваем ответ AI с ожидаемым ответом и присваиваем оценку.

In [12]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.5 to the AI assistant's response.

The response is partially aligned with the true response, as it correctly identifies the main goal of Explainable AI (XAI) as making AI systems more transparent and understandable. However, it lacks some key details and nuances present in the true response. For example, the true response mentions the importance of assessing fairness and accuracy, which is not explicitly mentioned in the AI assistant's response. Additionally, the true response uses more precise language, such as "providing insights into how they make decisions," which is not present in the AI assistant's response.
